In [1]:
import os
import pickle
import sys
import time

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.metrics import r2_score
from tensorflow import keras
from tensorflow.keras import layers
from tqdm.auto import tqdm, trange

tqdm.pandas()

from collections import defaultdict



In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
tf.config.list_physical_devices('GPU')

In [2]:
local_root_path = "."
sys.path.append(local_root_path)

In [13]:
import annutils

In [16]:
import importlib
importlib.reload(annutils)


<module 'annutils' from 'C:\\Users\\rripken\\Documents\\SalinityMLWorkshop_DMS_UCD-main\\annutils.py'>

Basic Setup

In [34]:

# Make a dir named Experiments
if not os.path.exists("Experiments"):
    os.mkdir("Experiments")

num_sheets = 9

observed_stations_ordered_by_median = ['RSMKL008', 'RSAN032', 'RSAN037', 'RSAC092', 'SLTRM004', 'ROLD024',
                                       'CHVCT000', 'RSAN018', 'CHSWP003', 'CHDMC006', 'SLDUT007', 'RSAN072',
                                       'OLD_MID', 'RSAN058', 'ROLD059', 'RSAN007', 'RSAC081', 'SLMZU025',
                                       'RSAC075', 'SLMZU011', 'SLSUS012', 'SLCBN002', 'RSAC064']

output_stations = ['CHDMC006-CVP INTAKE', 'CHSWP003-CCFB_INTAKE', 'CHVCT000-VICTORIA INTAKE',
                   'OLD_MID-OLD RIVER NEAR MIDDLE RIVER', 'ROLD024-OLD RIVER AT BACON ISLAND',
                   'ROLD059-OLD RIVER AT TRACY BLVD', 'RSAC064-SACRAMENTO R AT PORT CHICAGO',
                   'RSAC075-MALLARDISLAND', 'RSAC081-COLLINSVILLE', 'RSAC092-EMMATON',
                   'RSAC101-SACRAMENTO R AT RIO VISTA', 'RSAN007-ANTIOCH', 'RSAN018-JERSEYPOINT',
                   'RSAN032-SACRAMENTO R AT SAN ANDREAS LANDING', 'RSAN037-SAN JOAQUIN R AT PRISONERS POINT',
                   'RSAN058-ROUGH AND READY ISLAND', 'RSAN072-SAN JOAQUIN R AT BRANDT BRIDGE',
                   'RSMKL008-S FORK MOKELUMNE AT TERMINOUS', 'SLCBN002-CHADBOURNE SLOUGH NR SUNRISE DUCK CLUB',
                   'SLDUT007-DUTCH SLOUGH', 'SLMZU011-MONTEZUMA SL AT BELDONS LANDING',
                   'SLMZU025-MONTEZUMA SL AT NATIONAL STEEL', 'SLSUS012-SUISUN SL NEAR VOLANTI SL',
                   'SLTRM004-THREE MILE SLOUGH NR SAN JOAQUIN R', 'SSS-STEAMBOAT SL', 'CCW-MIDDLE RIVER INTAKE',
                   'OH4-OLD R @ HWY 4', 'SLRCK005-CCWD_Rock', 'MRU-MIDDLE RIVER AT UNDINE ROAD', 'HLL-HOLLAND TRACT',
                   'BET-PIPER SLOUGH @ BETHEL TRACT', 'GES-SACRAMENTO R BELOW GEORGIANA SLOUGH',
                   'NMR: N FORK MOKELUMNE R NEAR WALNUT GROVE', 'IBS-CORDELIA SLOUGH @ IBIS CLUB',
                   'GYS-GOODYEAR SLOUGH AT MORROW ISLAND CLUB', 'BKS-SLBAR002-North Bay Aqueduct/Barker Sl']

output_stations, name_mapping = annutils.read_output_stations(output_stations, observed_stations_ordered_by_median)

### Experiment:6 Years
This is just the base data for the 6 selected years.
 If we do this right the training data should be approx 6 * 365 =~ 2190 rows of training data


In [6]:
# Make a dir named 6years
experiment_name = "6years"
if not os.path.exists("Experiments/" + experiment_name):
    os.mkdir("Experiments/" + experiment_name)

picked_training_years = [
    ('2007-10-1','2008-9-30'),
    ('2008-10-1','2009-9-30'),
    ('2010-10-1','2011-9-30'),
    ('2011-10-1','2012-9-30'),
    ('2013-10-1','2014-9-30'),
    ('2016-10-1','2017-9-30')
]

input_files = ["dsm2_ann_inputs_base.xlsx"]

X_df= None
Y_df= None

for data_file in input_files:
    data_path = os.path.join(local_root_path,data_file)
    dfinps, dfouts = annutils.read_and_split(data_path, num_sheets, observed_stations_ordered_by_median)
    X_df = pd.concat([X_df, dfinps], axis=0)
    Y_df = pd.concat([Y_df, dfouts], axis=0)

# now X_df should have 8 input features and Y_df should have 23 target salinity values






In [20]:
ndays=118
window_size=0
nwindows=0
df_plus = annutils.create_antecedent_inputs(X_df,ndays=ndays,window_size=window_size,nwindows=nwindows)
# df_plus should now have 118 * 8 = 944 input features

# synchronize trims off the na values so the row numbers go from 10920 to 10803
df_X2, df_Y2 = annutils.synchronize(df_plus, Y_df)

train_X = annutils.include(df_X2, picked_training_years)
train_Y = annutils.include(df_Y2, picked_training_years)

test_X = annutils.exclude(df_X2, picked_training_years)
test_Y = annutils.exclude(df_Y2, picked_training_years)

train_X.to_csv(os.path.join("Experiments", experiment_name, "train_X.csv"))
train_Y.to_csv(os.path.join("Experiments", experiment_name, "train_Y.csv"))
test_X.to_csv(os.path.join("Experiments", experiment_name, "test_X.csv"))
test_Y.to_csv(os.path.join("Experiments", experiment_name, "test_Y.csv"))



### Experiment: 6 Years with Augmented data
This is the same as the previous experiment but adds in the augmented data.
The augmented data:
    sac + 15 days
    sac - 15 days
    sjr + 15 days
    sjr - 15 days
    sac + 20%
    sac - 20%
    sjr + 20%
    sjr - 20%

 If we do this right the training data should be approx 6 * 365 * 9 =~ 19710 rows of data

In [28]:
# Make a dir named 6years
experiment_name = "6yearsAugmented"
if not os.path.exists("Experiments/" + experiment_name):
    os.mkdir("Experiments/" + experiment_name)

picked_training_years = [
    ('2007-10-1','2008-9-30'),
    ('2008-10-1','2009-9-30'),
    ('2010-10-1','2011-9-30'),
    ('2011-10-1','2012-9-30'),
    ('2013-10-1','2014-9-30'),
    ('2016-10-1','2017-9-30')
]

input_files = ["dsm2_ann_inputs_base.xlsx",
               "dsm2_ann_inputs_rsacminus15day.xlsx",
               "dsm2_ann_inputs_rsacminus20pct.xlsx",
               "dsm2_ann_inputs_rsacplus15day.xlsx",
               "dsm2_ann_inputs_rsacplus20pct.xlsx",
               "dsm2_ann_inputs_rsanminus15day.xlsx",
               "dsm2_ann_inputs_rsanminus20pct.xlsx",
               "dsm2_ann_inputs_rsanplus15day.xlsx",
               "dsm2_ann_inputs_rsanplus20pct.xlsx"]

X_df= None
Y_df= None

ndays = 118
window_size = 0
nwindows = 0

for data_file in input_files:
    data_path = os.path.join(local_root_path,data_file)
    dfinps, dfouts = annutils.read_and_split(data_path, num_sheets, observed_stations_ordered_by_median)
    dfinps = annutils.create_antecedent_inputs(dfinps,ndays=ndays,window_size=window_size,nwindows=nwindows)
    dfinps, dfouts = annutils.synchronize(dfinps, dfouts)
    X_df = pd.concat([X_df, dfinps], axis=0)
    Y_df = pd.concat([Y_df, dfouts], axis=0)

# now X_df should have 118 * 8 = 944 input features and Y_df should have 23 target salinity values


train_X = annutils.include(X_df, picked_training_years)
train_Y = annutils.include(Y_df, picked_training_years)

test_X = annutils.exclude(X_df, picked_training_years)
test_Y = annutils.exclude(Y_df, picked_training_years)

train_X.to_csv(os.path.join("Experiments", experiment_name, "train_X.csv"))
train_Y.to_csv(os.path.join("Experiments", experiment_name, "train_Y.csv"))
test_X.to_csv(os.path.join("Experiments", experiment_name, "test_X.csv"))
test_Y.to_csv(os.path.join("Experiments", experiment_name, "test_Y.csv"))



### Experiment: Colab standard
This is how the Colab notebook builds the datasets.



In [32]:
experiment_name = "colab"
if not os.path.exists("Experiments/" + experiment_name):
    os.mkdir("Experiments/" + experiment_name)

train_data = ["dsm2_ann_inputs_rsacminus15day.xlsx",
              "dsm2_ann_inputs_rsacminus20pct.xlsx",
              "dsm2_ann_inputs_rsacplus15day.xlsx",
              "dsm2_ann_inputs_rsacplus20pct.xlsx",
              "dsm2_ann_inputs_rsanminus15day.xlsx",
              "dsm2_ann_inputs_rsanminus20pct.xlsx",
              "dsm2_ann_inputs_rsanplus15day.xlsx",
              "dsm2_ann_inputs_rsanplus20pct.xlsx",
              ]

test_data = {'dcc0': "dsm2_ann_inputs_dcc0.xlsx",
             'smscg1': "dsm2_ann_inputs_smscg1.xlsx",
             'dcc1': "dsm2_ann_inputs_dcc1.xlsx",
             'smscg0': "dsm2_ann_inputs_smscg0.xlsx"}

extra_data = {'observed': "observed_data_daily.xlsx"}
which_part_for_test = 'last'
extra_data_test_ratio = 0.3

ndays = 118
window_size = 0
nwindows = 0

xscaler = None
yscaler = None

def read_training_data(train_data):
    x_train = None
    y_train = None
    for data_file in tqdm(train_data):
        data_path = os.path.join(local_root_path, data_file)
        dfinps, dfouts = annutils.read_and_split(data_path, num_sheets, observed_stations_ordered_by_median)

        # create tuple of calibration and validation sets and the xscaler and yscaler on the combined inputs
        if x_train is None:
            (x_train, y_train), (_, _), _, _ = \
                annutils.create_training_sets([dfinps],
                                              [dfouts],
                                              train_frac=1,
                                              ndays=ndays, window_size=window_size, nwindows=nwindows,
                                              xscaler=xscaler, yscaler=yscaler)
        else:
            (xc, yc), (_, _), _, _ = \
                annutils.create_training_sets([dfinps],
                                              [dfouts],
                                              train_frac=1,
                                              ndays=ndays, window_size=window_size, nwindows=nwindows,
                                              xscaler=xscaler, yscaler=yscaler)
            x_train = pd.concat([x_train, xc], axis=0)
            y_train = pd.concat([y_train, yc], axis=0)
            del xc, yc
    return x_train, y_train

train_X, train_Y = read_training_data(train_data)


test_X, test_Y = read_training_data(test_data.values())


######### Read extra observed dataset ###############
for data_file in tqdm(extra_data.values()):
    data_path = os.path.join(local_root_path, data_file)

    # print("Starting read_excel calls:", data_path)
    dflist = [annutils.read_excel_sheet(data_path, i) for i in range(num_sheets)]

    df_inpout = pd.concat(dflist[0:num_sheets], axis=1).dropna(axis=0)
    col_mask = df_inpout.columns.isin(dflist[num_sheets - 1].columns)
    dfinps = df_inpout.loc[:, ~col_mask]
    dfouts = df_inpout.loc[:, col_mask]
    # dfouts = dfouts[output_stations]  # out_stations is None here...

    start_year = max(dfinps.index[0].year, dfouts.index[0].year)
    end_year = min(dfinps.index[-1].year, dfouts.index[-1].year)

    if which_part_for_test == 'last':
        calib_slice = slice(str(start_year),
                            str(int(start_year + (1 - extra_data_test_ratio) * (end_year - start_year))))
        valid_slice = slice(str(int(start_year + (1 - extra_data_test_ratio) * (end_year - start_year)) + 1),
                            str(end_year))
    elif which_part_for_test == 'first':
        calib_slice = slice(str(int(start_year + (1 - extra_data_test_ratio) * (end_year - start_year)) + 1),
                            str(end_year))
        valid_slice = slice(str(start_year),
                            str(int(start_year + (1 - extra_data_test_ratio) * (end_year - start_year))))
    elif which_part_for_test == 'middle':
        calib_slice = [slice(str(start_year),
                             str(int(start_year + (1 - extra_data_test_ratio) / 2 * (end_year - start_year)))),
                       slice(str(int(start_year + (1 + extra_data_test_ratio) / 2 * (end_year - start_year) + 1)),
                             str(end_year))]
        valid_slice = slice(str(int(start_year + (1 - extra_data_test_ratio) / 2 * (end_year - start_year)) + 1),
                            str(int(start_year + (1 + extra_data_test_ratio) / 2 * (end_year - start_year))))
    elif which_part_for_test == 'manual' and picked_training_years is not None:
        calib_slice = [slice(str(start_year), str(end_year)) for (start_year, end_year) in picked_training_years]
        valid_slice = [slice(start_year, end_year) for ((_, start_year), (end_year, _)) in
                       zip([(None, '1989-10-1'), ] + picked_training_years,
                           picked_training_years + [('2020-9-30', None), ])]
    else:
        raise Exception('Unknown data splitting method')

    # create tuple of calibration and validation sets and the xscaler and yscaler on the combined inputs
    (x_extra_train, y_extra_train), (x_extra_test, y_extra_test), _, _ = \
        annutils.create_training_sets([dfinps],
                                      [dfouts],
                                      calib_slice=calib_slice,
                                      valid_slice=valid_slice,
                                      ndays=ndays, window_size=window_size, nwindows=nwindows,
                                      xscaler=xscaler, yscaler=yscaler)
    train_X = pd.concat([train_X, x_extra_train], axis=0)
    train_Y = pd.concat([train_Y, y_extra_train], axis=0)
    test_X = pd.concat([test_X, x_extra_test], axis=0)
    test_Y = pd.concat([test_Y, y_extra_test], axis=0)



  0%|          | 0/8 [00:00<?, ?it/s]

Randomly selecting 10803 samples for training, 0 for test
Randomly selecting 10803 samples for training, 0 for test
Randomly selecting 10803 samples for training, 0 for test
Randomly selecting 10803 samples for training, 0 for test
Randomly selecting 10803 samples for training, 0 for test
Randomly selecting 10803 samples for training, 0 for test
Randomly selecting 10803 samples for training, 0 for test
Randomly selecting 10803 samples for training, 0 for test


  0%|          | 0/4 [00:00<?, ?it/s]

Randomly selecting 0 samples for training, 0 for test
Randomly selecting 10803 samples for training, 0 for test
Randomly selecting 0 samples for training, 0 for test
Randomly selecting 10803 samples for training, 0 for test


  0%|          | 0/1 [00:00<?, ?it/s]

In [33]:
train_X.to_csv(os.path.join("Experiments", experiment_name, "train_X.csv"))
train_Y.to_csv(os.path.join("Experiments", experiment_name, "train_Y.csv"))
test_X.to_csv(os.path.join("Experiments", experiment_name, "test_X.csv"))
test_Y.to_csv(os.path.join("Experiments", experiment_name, "test_Y.csv"))